In [1]:
! pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=487fd96d35d9ccd2ed30de81977d590eb84bbf8339181e25c7f2a66c2c0ed409
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# Importing necessary Dependencies/Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkFiles , SparkContext
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv("All.csv")

df.head()

,sector,Name,symbol,code,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,...,EV / EBIT,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification
0,Academic & Education,Global Education Communities,GEC,A&E 01,2013,0.63,0.61,0.20,-0.10,0.63,...,-6.2x,-4.0x,0.9x,0.9x,-5.9x,1.1x,NaN,-3.1,0,0
1,Academic & Education,Global Education Communities,GEC,A&E 01,2014,0.78,0.76,0.16,-0.25,0.78,...,-9.0x,-2.6x,0.7x,0.5x,-14.3x,1.0x,NaN,-0.9,6,0
2,Academic & Education,Global Education Communities,GEC,A&E 01,2015,0.27,0.26,0.05,0.08,0.27,...,-22.7x,16.6x,1.0x,1.9x,-18.7x,1.1x,NaN,-0.7,4,0
3,Academic & Education,Global Education Communities,GEC,A&E 01,2016,0.76,0.73,0.27,-0.04,0.76,...,54.3x,84.6x,1.1x,2.4x,6.4x,1.5x,NaN,2.9,4,1
4,Academic & Education,Global Education Communities,GEC,A&E 01,2017,0.32,0.31,0.12,0.18,0.32,...,162.1x,28.4x,1.2x,3.4x,30.6x,1.8x,NaN,1.9,4,1


In [14]:
# Replacing all the null values with 0
df.fillna(0, inplace=True)

# Replacing x suffix from the DataFrame

columns_to_clean = ['EV/EBITDA', 'EV / EBIT', 'EV / Free Cash Flow',
                    'EV / Invested Capital', 'EV / Revenue', 'P/E Ratio', 'Price/Book']
df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: x.rstrip('x') if isinstance(x, str) else x)

df.head()


,sector,Name,symbol,code,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,...,EV / EBIT,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification
0,Academic & Education,Global Education Communities,GEC,A&E 01,2013,0.63,0.61,0.20,-0.10,0.63,...,-6.2,-4.0,0.9,0.9,-5.9,1.1,0.0,-3.1,0,0
1,Academic & Education,Global Education Communities,GEC,A&E 01,2014,0.78,0.76,0.16,-0.25,0.78,...,-9.0,-2.6,0.7,0.5,-14.3,1.0,0.0,-0.9,6,0
2,Academic & Education,Global Education Communities,GEC,A&E 01,2015,0.27,0.26,0.05,0.08,0.27,...,-22.7,16.6,1.0,1.9,-18.7,1.1,0.0,-0.7,4,0
3,Academic & Education,Global Education Communities,GEC,A&E 01,2016,0.76,0.73,0.27,-0.04,0.76,...,54.3,84.6,1.1,2.4,6.4,1.5,0.0,2.9,4,1
4,Academic & Education,Global Education Communities,GEC,A&E 01,2017,0.32,0.31,0.12,0.18,0.32,...,162.1,28.4,1.2,3.4,30.6,1.8,0.0,1.9,4,1


In [15]:
df.dtypes

sector                                object
Name                                  object
symbol                                object
code                                  object
year                                   int64
Current ratio                         object
Quick Ratio                           object
Cash ratio                            object
Operating cash flow ratio             object
Working capital ratio                 object
Return On Equity (ROE)                object
Net profit margin                     object
Sales turnover ratio                  object
Return On Assets (ROA)                object
EPS                                  float64
Debt-to-assets ratio                  object
Equity Ratio                          object
Debt to equity ratio                  object
Interest coverage ratio               object
The Long-Term Debt Coverage Ratio     object
Total debt to EBITDA ratio            object
Cash flow to debt ratio               object
Inventory 